In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.stattools import durbin_watson
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [5]:
# Load the merged dataset
merged_data_path = 'C:/Users/Administrator/Documents/kifiya/Week_10/data/Merged_brent_oil_prices_with_Indicators.csv'
data = pd.read_csv(merged_data_path, parse_dates=['Date'])
data.set_index('Date', inplace=True)

In [7]:
# Step 1: Drop non-numeric columns and handle missing values
data = data.select_dtypes(include=[np.number]).dropna()

# Step 2: Define a function to check stationarity
# Function to check stationarity of a time series
def check_stationarity(timeseries):
    result = adfuller(timeseries.dropna())
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    return result[1] <= 0.05  # Returns True if series is stationary

# Modified function to make the dataset stationary, with handling for constant columns
def make_stationary(df):
    for column in df.columns:
        # Check if the column is constant
        if df[column].nunique() == 1:
            print(f"Column '{column}' is constant and will be skipped.")
            continue
        
        # Apply differencing until the series becomes stationary
        diff_count = 0
        while not check_stationarity(df[column]):
            df[column] = df[column].diff().dropna()
            diff_count += 1
            if diff_count > 2:  # Prevent infinite loop, max 2 differences
                print(f"{column} required more than 2 differencing steps to become stationary.")
                break
    return df

# Run the modified make_stationary function on a copy of the data
data_stationary = make_stationary(data.copy())


# Train-test split for VAR
train_size = int(len(data_stationary) * 0.8)
train_var, test_var = data_stationary[:train_size], data_stationary[train_size:]

ADF Statistic: -1.6520
p-value: 0.4560
ADF Statistic: -11.3732
p-value: 0.0000
ADF Statistic: -11.0620
p-value: 0.0000
ADF Statistic: -11.0337
p-value: 0.0000
ADF Statistic: -3.1405
p-value: 0.0237
ADF Statistic: -3.6617
p-value: 0.0047
ADF Statistic: -6.3901
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic: -0.9303
p-value: 0.7778
ADF Statistic: -21.7025
p-value: 0.0000
ADF Statistic

In [15]:
# Step 4: Manually select the best lag order by fitting VAR models with different lags
from statsmodels.tsa.api import VAR

# Define a range of lags to test
max_lag = 15  # You can adjust this based on your dataset's length
aic_values = []
bic_values = []

# Iterate over each lag order and calculate the AIC and BIC
for lag in range(1, max_lag + 1):
    try:
        model = VAR(train_var)
        model_fitted = model.fit(lag)
        aic_values.append((lag, model_fitted.aic))
        bic_values.append((lag, model_fitted.bic))
        print(f"Lag: {lag}, AIC: {model_fitted.aic}, BIC: {model_fitted.bic}")
    except Exception as e:
        print(f"Error fitting VAR model with lag {lag}: {e}")
        continue

# Select the lag with the lowest AIC
best_aic_lag = min(aic_values, key=lambda x: x[1])[0]
best_bic_lag = min(bic_values, key=lambda x: x[1])[0]
print(f"Optimal lag by AIC: {best_aic_lag}")
print(f"Optimal lag by BIC: {best_bic_lag}")

# Use the best lag based on AIC (or BIC) to fit the final model
selected_lag = best_aic_lag  # Choose either AIC or BIC
model_fitted = VAR(train_var).fit(selected_lag)
print(f"VAR model fitted with lag order {selected_lag}.")


c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency in

Error fitting VAR model with lag 1: x contains one or more constant columns. Column(s) 76, 97, 121, 145, 185, 208, 225, 230, 239, 261, 263, 266, 269, 278, 285, 292, 304, 311, 312, 317, 333, 379, 393, 420, 422, 425, 450, 453, 461, 482, 483, 486, 492, 502, 506, 509, 514, 517, 519, 520, 521, 522, 524, 526, 527, 529, 531, 543, 550, 555, 556, 559, 560, 562, 564, 566, 567, 571, 573, 574, 579, 582, 583, 584, 586, 588, 596, 597, 598, 601, 604, 613, 614, 618, 619, 622, 626, 627, 630, 636, 640, 648, 650, 651, 657, 658, 665, 671, 672, 673, 675, 678, 679, 680, 681, 682, 689, 691, 697, 700, 702, 706 are constant. Adding a constant with trend='c' is not allowed.
Error fitting VAR model with lag 2: x contains one or more constant columns. Column(s) 76, 97, 121, 145, 185, 208, 225, 230, 239, 261, 263, 266, 269, 278, 285, 292, 304, 311, 312, 317, 333, 379, 393, 420, 422, 425, 450, 453, 461, 482, 483, 486, 492, 502, 506, 509, 514, 517, 519, 520, 521, 522, 524, 526, 527, 529, 531, 543, 550, 555, 556, 559

c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Error fitting VAR model with lag 6: x contains one or more constant columns. Column(s) 76, 97, 121, 145, 185, 208, 225, 230, 239, 261, 263, 266, 269, 278, 285, 292, 304, 311, 312, 317, 333, 379, 393, 420, 422, 425, 450, 453, 461, 482, 483, 486, 492, 502, 506, 509, 514, 517, 519, 520, 521, 522, 524, 526, 527, 529, 531, 543, 550, 555, 556, 559, 560, 562, 564, 566, 567, 571, 573, 574, 579, 582, 583, 584, 586, 588, 596, 597, 598, 601, 604, 613, 614, 618, 619, 622, 626, 627, 630, 636, 640, 648, 650, 651, 657, 658, 665, 671, 672, 673, 675, 678, 679, 680, 681, 682, 689, 691, 697, 700, 702, 706, 787, 808, 832, 856, 896, 919, 936, 941, 950, 972, 974, 977, 980, 989, 996, 1003, 1015, 1022, 1023, 1028, 1044, 1090, 1104, 1131, 1133, 1136, 1161, 1164, 1172, 1193, 1194, 1197, 1203, 1213, 1217, 1220, 1225, 1228, 1230, 1231, 1232, 1233, 1235, 1237, 1238, 1240, 1242, 1254, 1261, 1266, 1267, 1270, 1271, 1273, 1275, 1277, 1278, 1282, 1284, 1285, 1290, 1293, 1294, 1295, 1297, 1299, 1307, 1308, 1309, 1312, 

c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


...

c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Error fitting VAR model with lag 14: x contains one or more constant columns. Column(s) 76, 97, 121, 145, 185, 208, 225, 230, 239, 261, 263, 266, 269, 278, 285, 292, 304, 311, 312, 317, 333, 379, 393, 420, 422, 425, 450, 453, 461, 482, 483, 486, 492, 502, 506, 509, 514, 517, 519, 520, 521, 522, 524, 526, 527, 529, 531, 543, 550, 555, 556, 559, 560, 562, 564, 566, 567, 571, 573, 574, 579, 582, 583, 584, 586, 588, 596, 597, 598, 601, 604, 613, 614, 618, 619, 622, 626, 627, 630, 636, 640, 648, 650, 651, 657, 658, 665, 671, 672, 673, 675, 678, 679, 680, 681, 682, 689, 691, 697, 700, 702, 706, 787, 808, 832, 856, 896, 919, 936, 941, 950, 972, 974, 977, 980, 989, 996, 1003, 1015, 1022, 1023, 1028, 1044, 1090, 1104, 1131, 1133, 1136, 1161, 1164, 1172, 1193, 1194, 1197, 1203, 1213, 1217, 1220, 1225, 1228, 1230, 1231, 1232, 1233, 1235, 1237, 1238, 1240, 1242, 1254, 1261, 1266, 1267, 1270, 1271, 1273, 1275, 1277, 1278, 1282, 1284, 1285, 1290, 1293, 1294, 1295, 1297, 1299, 1307, 1308, 1309, 1312,

c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Error fitting VAR model with lag 15: x contains one or more constant columns. Column(s) 76, 97, 121, 145, 185, 208, 225, 230, 239, 261, 263, 266, 269, 278, 285, 292, 304, 311, 312, 317, 333, 379, 393, 420, 422, 425, 450, 453, 461, 482, 483, 486, 492, 502, 506, 509, 514, 517, 519, 520, 521, 522, 524, 526, 527, 529, 531, 543, 550, 555, 556, 559, 560, 562, 564, 566, 567, 571, 573, 574, 579, 582, 583, 584, 586, 588, 596, 597, 598, 601, 604, 613, 614, 618, 619, 622, 626, 627, 630, 636, 640, 648, 650, 651, 657, 658, 665, 671, 672, 673, 675, 678, 679, 680, 681, 682, 689, 691, 697, 700, 702, 706, 787, 808, 832, 856, 896, 919, 936, 941, 950, 972, 974, 977, 980, 989, 996, 1003, 1015, 1022, 1023, 1028, 1044, 1090, 1104, 1131, 1133, 1136, 1161, 1164, 1172, 1193, 1194, 1197, 1203, 1213, 1217, 1220, 1225, 1228, 1230, 1231, 1232, 1233, 1235, 1237, 1238, 1240, 1242, 1254, 1261, 1266, 1267, 1270, 1271, 1273, 1275, 1277, 1278, 1282, 1284, 1285, 1290, 1293, 1294, 1295, 1297, 1299, 1307, 1308, 1309, 1312,

ValueError: min() arg is an empty sequence

In [ ]:
# Step 5: Forecast using the VAR model
lagged_values = train_var.values[-lag_order.aic:]
forecast = model_fitted.forecast(y=lagged_values, steps=len(test_var))
forecast_df = pd.DataFrame(forecast, index=test_var.index, columns=test_var.columns)

In [ ]:
# Step 6: Plot VAR forecasting results
plt.figure(figsize=(12, 8))
for col in data_stationary.columns:
    plt.plot(train_var.index, train_var[col], label=f'Train: {col}')
    plt.plot(test_var.index, test_var[col], label=f'Test: {col}')
    plt.plot(forecast_df.index, forecast_df[col], label=f'Forecast: {col}')
plt.title('VAR Model Forecasting')
plt.xlabel('Date')
plt.ylabel('Values')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Step 7: Prepare data for LSTM model
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.values)

def create_sequences(data, look_back=60):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, :])
        y.append(data[i + look_back, 0])  # Target is Brent oil price
    return np.array(X), np.array(y)

look_back = 60
X, y = create_sequences(data_scaled, look_back)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Split data for LSTM
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_size = X.shape[2]
hidden_size = 50
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train LSTM model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 1 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:

# LSTM predictions
model.eval()
with torch.no_grad():
    test_pred = model(X_test).numpy()
    test_pred = scaler.inverse_transform(np.concatenate([test_pred, np.zeros((test_pred.shape[0], data.shape[1] - 1))], axis=1))[:, 0]
    y_test_actual = scaler.inverse_transform(np.concatenate([y_test.numpy().reshape(-1, 1), np.zeros((y_test.shape[0], data.shape[1] - 1))], axis=1))[:, 0]


In [ ]:
# Plotting VAR vs LSTM predictions
plt.figure(figsize=(14, 7))
plt.plot(test_var.index, test_var['Price'], label='Actual Brent Price (VAR)', color='blue')
plt.plot(forecast_df.index, forecast_df['Price'], label='VAR Forecast', color='orange')
plt.plot(test_var.index, y_test_actual, label='Actual Brent Price (LSTM)', color='green')
plt.plot(test_var.index, test_pred, label='LSTM Forecast', color='red')
plt.title('Brent Oil Price Forecast: VAR vs LSTM')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()


KeyboardInterrupt: 